Stap 1 – Data importeren en opschonen

In [32]:
import pandas as pd

# Dataset inladen
df = pd.read_csv("Heerlen_dataset.csv")

# Verwijder rijen zonder cruciale informatie
df = df.dropna(subset=['bouwjaar', 'opp_pand', 'lon', 'lat'])

# Zorg dat de kolommen numeriek zijn voor berekeningen
df['bouwjaar'] = pd.to_numeric(df['bouwjaar'], errors='coerce')
df['opp_pand'] = pd.to_numeric(df['opp_pand'], errors='coerce')

# Voeg een extra kolom toe met het volledige adres als één string
df['adres'] = df['openbareruimtenaam'].fillna('') + ' ' + df['huisnummer'].astype(str)

Stap 2 – Twinbuilding-voorspelfunctie definiëren

Doel: vergelijk een gekozen gebouw met de rest van de dataset op basis van:
* Bouwjaar (± 5 jaar)
* Oppervlakte (± 10%)
* Gelijk woningtype
* Zelfde gebruiksfuncties (zoals winkel, kantoor, woon)

In [33]:
def find_twins(ref, df, buffer_years=5, surface_tolerance=0.1):
    matches = df[
        (df['pand_id'] != ref['pand_id']) &  # Niet zichzelf vergelijken
        (abs(df['bouwjaar'] - ref['bouwjaar']) <= buffer_years) &
        (abs(df['opp_pand'] - ref['opp_pand']) / ref['opp_pand'] <= surface_tolerance) &
        (df['woningtype'] == ref['woningtype']) &
        (df[['woonfunctie', 'kantoorfunctie', 'winkelfunctie']].eq(
            ref[['woonfunctie', 'kantoorfunctie', 'winkelfunctie']]
        ).all(axis=1))
    ]
    return matches

Stap 3 – Interactieve kaartweergave maken met folium

Doel: toon de referentie en gevonden twins op een kaart.
* Rood icoon = referentiegebouw
* Blauwe iconen = mogelijke twinbuildings
* Toon extra informatie bij klikken op marker (popup)

In [34]:
import folium
from folium.plugins import MarkerCluster
from IPython.display import display
from ipywidgets import Output

out = Output()

def show_map_by_adres(adres):
    out.clear_output()
    with out:
        ref = df[df['adres'] == adres].iloc[0]          # Gebruikerselectie
        twins = find_twins(ref, df)                     # Twinbuilding zoekactie

        m = folium.Map(location=[ref['lat'], ref['lon']], zoom_start=16)
        MarkerCluster().add_to(m)

        # Marker voor referentiegebouw
        folium.Marker(
            [ref['lat'], ref['lon']],
            popup=f"<b>Referentie</b><br>{ref['adres']}<br>{int(ref['bouwjaar'])}, {int(ref['opp_pand'])} m²",
            icon=folium.Icon(color="red", icon="home")
        ).add_to(m)

        # Markers voor twins
        for _, r in twins.iterrows():
            folium.Marker(
                [r['lat'], r['lon']],
                popup=f"{r['adres']}<br>{int(r['bouwjaar'])}, {int(r['opp_pand'])} m²",
                icon=folium.Icon(color="blue", icon="home")
            ).add_to(m)

        display(m)

        # Toon resultaten als tabel
        print(f"\nGevonden twinbuildings: {len(twins)}")
        display(twins[['adres', 'bouwjaar', 'opp_pand', 'woningtype']].drop_duplicates().reset_index(drop=True))

Stap 4 – Interactieve gebruikersinterface maken

Doel: laat gebruiker zelf adres selecteren via dropdown (of later zoekfunctie toevoegen).
* Zodra iets geselecteerd is, roep je automatisch de kaartfunctie aan.
* Output verschijnt direct onder de selectie.

In [ ]:
from ipywidgets import interact, Dropdown

interact(
    show_map_by_adres,
    adres=Dropdown(
        options=sorted(df['adres'].unique()),
        description="Adres:"
    )
)
display(out)

interactive(children=(Dropdown(description='Adres:', options=('A gen Giezen 1', 'A gen Giezen 10', 'A gen Giez…

Output()